<a href="https://colab.research.google.com/github/javst/MachineVision/blob/main/CRIFT10%EF%BC%882%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

batchsz = 32
cifar_train = datasets.CIFAR10('cifar',True,transform=transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
]),download=True)

cifar_train = DataLoader(cifar_train,batch_size=batchsz,shuffle=True)
cifar_test = datasets.CIFAR10('cifar',True,transform=transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
]),download=True)

cifar_test = DataLoader(cifar_test,batch_size=batchsz,shuffle=True)

x,label = iter(cifar_train).next()
print('x:',x.shape,'label',label.shape)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting cifar/cifar-10-python.tar.gz to cifar
Files already downloaded and verified
x: torch.Size([32, 3, 32, 32]) label torch.Size([32])


In [2]:
import torch
from torch import nn
from torch.nn import functional as F
class Lenet5(nn.Module):
    #for cifar 10 dataset
    def __init__(self):
        super(Lenet5,self).__init__()
        self.conv_unit = nn.Sequential(
            #x:[b,3,32,32]=>[b,6,]
            nn.Conv2d(3,6,kernel_size=5,stride=1,padding=0),
            nn.AvgPool2d(kernel_size=2,stride=2,padding=0),
            #
            nn.Conv2d(6,16,kernel_size=5,stride=1,padding=0),
            nn.AvgPool2d(kernel_size=2,stride=2,padding=0),
            #
            
        )
        #flatten
        #fcunitl
        self.fc_unit = nn.Sequential(
            nn.Linear(16*5*5,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10)
        )

        #self.criteon = nn.CrossEntropyLoss()
        
    def forward(self,x):
        batchsz=x.size(0)
        x = self.conv_unit(x)
        b=x.view(batchsz,16*5*5)
        logits = self.fc_unit(b)
        return logits
        #pred = F.softmax(logits,dim=1)
        #loss = self.criteon(logits,y)

In [ ]:
from torch import optim
device = torch.device('cuda')
model = Lenet5().to(device)
criteon = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)
for epoch in range(300):
    for batchidx,(x,label) in enumerate(cifar_train):
        x,label = x.to(device),label.to(device)
        logits = model(x)
        loss = criteon(logits,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batchidx%10 == 0 :
           print(epoch,loss.item())
        
    
     
        
        

0 2.2908713817596436
0 2.2885429859161377
0 2.3060007095336914
0 2.2880356311798096
0 2.2882630825042725
0 2.190497636795044
0 2.2159242630004883
0 2.0208230018615723
0 2.149778127670288
0 2.0709521770477295
0 2.1521213054656982
0 2.349717617034912
0 2.1123709678649902
0 2.2666995525360107
0 1.9374639987945557
0 1.8417789936065674
0 2.039212703704834
0 1.850964069366455
0 2.133646249771118
0 2.0853798389434814
0 2.004072904586792
0 2.227449893951416
0 2.0517518520355225
0 1.9684135913848877
0 1.935082197189331
0 1.9426512718200684
0 2.2937092781066895
0 1.7707597017288208
0 2.0485472679138184
0 1.9615625143051147
0 1.7071027755737305
0 1.9197537899017334
0 1.9667733907699585
0 1.8766742944717407
0 1.6725572347640991
0 1.6604629755020142
0 1.9191861152648926
0 1.8003495931625366
0 1.840118408203125
0 2.018918752670288
0 1.4557167291641235
0 1.8128589391708374
0 1.9460595846176147
0 1.7867581844329834
0 1.727340579032898
0 1.7735412120819092
0 1.7485774755477905
0 1.6708844900131226
0 1.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
total_correct = 0
total_num =0
with torch.no_grad():
  for x,label in cifar_test:
    x,label = x.to(device),label.to(device)
    logits = model(x)
    pred = logits.argmax(dim = 1)
    total_correct +=torch.eq(pred,label).float().sum()
    total_num +=x.size(0)
acc = total_corretc/total_num
print(epoch,acc)

NameError: ignored